## Práctica de vecinos más cercanos (continuación)

## Objetivos formativos
- Emplear diferentes métricas
- Normalización de datos
- Emplear PCA para reducir dimensionalidad de los datos y potencialmente mejorar resultados
- Introducir el concepto de Pipeline en sklearn
- Emplear los k-vecinos más cercanos como regresor 

## Métricas

La función de distancia empleada por defecto es la distancia euclídea. Pero podríamos probar diferentes métricas:



In [ ]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()
print(digits.images.shape)

n_samples = len(digits.images)
X=digits.images.reshape(n_samples,-1)
Y=digits.target

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=12)

# Creamos un diccionario con todos los parámetros a modificar
G = {"n_neighbors":range(1,10,2), "metric":["l2","l1"]}  

GS = GridSearchCV(KNeighborsClassifier(), G, scoring='accuracy', refit=True, cv=5) # validación cruzada de 5 particiones

knn = GS.fit(X_train, y_train)
print(GS.cv_results_["params"])
print(GS.cv_results_["mean_test_score"])

acc= knn.score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')


## Normalización y PCA

Además KNN podría beneficiarse de una normaliación de los datos y una proyección mediante PCA con el fin de reducir la dimensionalidad y potencialmente mejorar los resultados. Para ello vamos a implementar un PipeLine preproceso-clasificador:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline;

scaler = StandardScaler() # x=(x-media)/desviación_típica
pca = PCA() # parámetro n_components que puede tomar valores en [1,dim] en este caso: [1,64]
knn = KNeighborsClassifier()

# definimos un pipeline
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("knn", knn)])

# definimos un grid search donde modificamos parámetros de los
# diferentes componentes del pipeline:
G = {"pca__n_components": [4,8,16,24,32,40], "knn__n_neighbors": [1,3,4,5]}

GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

### Ejercicio

Modifica los parámetros que creas necesarios para obtener el mejor acierto posible

## KNN para **regresión**

Para ello emplearemos el dataset California Housing Dataset. Dicho dataset es un conjunto de datos clásico en el ámbito de la regresión, utilizado principalmente para predecir el precio medio de las viviendas en distintos distritos de California. Estos datos fueron extraídos originalmente del censo de Estados Unidos en 1990, y su objetivo es proporcionar información relevante sobre las características demográficas y geográficas de los distritos que afectan al precio de las viviendas.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

data = fetch_california_housing()

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn_regressor = KNeighborsRegressor(n_neighbors=5)

knn_regressor.fit(X_train, y_train)

y_pred = knn_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Sample Predictions: {y_pred[:10]}")


### Ejercicio

Modifica los parámetros que creas necesarios para obtener el menor error posible. Emplea validación cruzada.